In [ ]:
import anndata
import scipy.sparse as sp
import numpy as np

# baron2016_h & Xin (human pancreas)

## data size and shape

In [ ]:
dataset_pancreas = {'baron':None, 'xin': None}
dataset_pancreas['baron'] = anndata.read_h5ad("./data/baron_2016h.h5ad")
dataset_pancreas['xin'] = anndata.read_h5ad("./data/xin_2016.h5ad")

for k in dataset_pancreas.keys():
    print ("There are {} cells and {} genes in {} dataset".format(dataset_pancreas[k].shape[0],\
                                                                  dataset_pancreas[k].shape[1],\
                                                                  k))
    
    

In [ ]:
selesct = select_genes(data=dataset_pancreas['baron'].X, n=3000, threshold=0, plot=False)

In [ ]:
1_000_000

In [ ]:
a.obs['labels'].unique().tolist()

In [ ]:
a.X.toarray().shape

## batch and cell label info

In [ ]:
import matplotlib.pyplot as plt
plt.figure(figsize=(12,6))
plt.subplot(1,2,1)
dataset_pancreas['baron'].obs['batch_id'].value_counts().plot(kind='bar')
plt.xticks(rotation='vertical')
plt.subplot(1,2,2)
dataset_pancreas['xin'].obs['batch_id'].value_counts().plot(kind='bar')
plt.xticks(rotation='vertical')

In [ ]:
plt.figure(figsize=(12,6))
plt.subplot(1,2,1)
dataset_pancreas['baron'].obs['labels'].value_counts().plot(kind='bar')
plt.subplot(1,2,2)
dataset_pancreas['xin'].obs['labels'].value_counts().plot(kind='bar')

# Hrvatin and Chen (Mouse Brain)

In [ ]:
dataset_mouse_brain = {'hrvatin':None, 'chen': None}
dataset_mouse_brain['hrvatin'] = anndata.read_h5ad("./data/hrvatin_2018.h5ad")
dataset_mouse_brain['chen'] = anndata.read_h5ad("./data/chen_2017.h5ad")

for k in dataset_mouse_brain.keys():
    print ("There are {} cells and {} genes in {} dataset".format(dataset_mouse_brain[k].shape[0],\
                                                                  dataset_mouse_brain[k].shape[1],\
                                                                  k))

## batch and cell label info

In [ ]:
import matplotlib.pyplot as plt
plt.figure(figsize=(12,6))
plt.subplot(1,2,1)
dataset_mouse_brain['hrvatin'].obs['batch_id'].value_counts().plot(kind='bar')
plt.xticks(rotation='vertical')
plt.subplot(1,2,2)
dataset_mouse_brain['chen'].obs['batch_id'].value_counts().plot(kind='bar')
plt.xticks(rotation='vertical')

In [ ]:
plt.figure(figsize=(12,6))
plt.subplot(1,2,1)
dataset_mouse_brain['hrvatin'].obs['labels'].value_counts().plot(kind='bar')
plt.subplot(1,2,2)
dataset_mouse_brain['chen'].obs['labels'].value_counts().plot(kind='bar')

# Macosko and Shekhar (Mouse Retina)

In [ ]:
dataset_mouse_retina = {'macosko':None, 'shekhar': None}
dataset_mouse_retina['macosko'] = anndata.read_h5ad("./data/macosko_2015.h5ad")
dataset_mouse_retina['shekhar'] = anndata.read_h5ad("./data/shekhar_2016.h5ad")

for k in dataset_mouse_retina.keys():
    print ("There are {} cells and {} genes in {} dataset".format(dataset_mouse_retina[k].shape[0],\
                                                                  dataset_mouse_retina[k].shape[1],\
                                                                  k))

In [ ]:
plt.figure(figsize=(12,6))
plt.subplot(1,2,1)
dataset_mouse_retina['macosko'].obs['batch_id'].value_counts().plot(kind='bar')
plt.xticks(rotation='vertical')
plt.subplot(1,2,2)
dataset_mouse_retina['shekhar'].obs['batch_id'].value_counts().plot(kind='bar')
plt.xticks(rotation='vertical')

In [ ]:
plt.figure(figsize=(12,6))
plt.subplot(1,2,1)
dataset_mouse_retina['macosko'].obs['labels'].value_counts().plot(kind='bar')
plt.subplot(1,2,2)
dataset_mouse_retina['shekhar'].obs['labels'].value_counts().plot(kind='bar')